# <center>Dynamic discrete choice</center>
### <center>Alfred Galichon (NYU & Sciences Po)</center>
## <center>'math+econ+code' masterclass series</center>
#### <center>With python code examples</center>
© 2018–2024 by Alfred Galichon. Past and present support from NSF grant DMS-1716489, ERC grant CoG-866274 are acknowledged, as well as inputs from contributors listed [here](http://www.math-econ-code.org/team).

**If you reuse material from this masterclass, please cite as:**<br>
Alfred Galichon, 'math+econ+code' masterclass series. https://www.math-econ-code.org/

### Learning Objectives

* Estimation of infinite-horizon dynamic discrete choice models

### References

* Ford Jr, L. R., & Fulkerson, D. R. (1958). Constructing maximal dynamic flows from static flows. *Operations research*, 6(3), 419-433.

* Howard, R. (1960). *Dynamic programming and Markov processes*. Wiley.

* Schrijver, A. (2003). *Combinatorial optimization: polyhedra and efficiency* Vol. A. Springer. Section 12.5.c.

* Bertsekas, D. (2011), *Dynamic Programming and Optimal Control*, Vols. I and II. 3rd ed. Athena.

* Ljungqvist, L., & Sargent, T. (2012), *Recursive Macroeconomic Theory* 3rd ed. MIT.

* Rust (1987), Optimal replacement of GMC bus engines: an empirical model of Harold Zurcher. *Econometrica*.

* Nazareth, J., & Kulkarni, R. (1986). Linear programming formulations of Markov decision processes. *Operations Research Letters*.

* Ranie Lin, Rust (1987) replication. Python project, available on github https://github.com/ranielin/Rust-1987-Replication. 



### Libraries

Let's start by loading the libraries we shall need for this course.

In [1]:
#!pip install nlopt # if from Colab
import numpy as np
import scipy.sparse as sp
import nlopt

## Loading the data

The following function loads the data from John Rust's files, cleans them and returns them as a `np.array`. It outputs a three column array whose:
 - first column is the milage range (per 5,000 miles brackets)
 - second column is the decision to replace (1) or not (0)
 - third column is number of additional mileage brackets after running one period 

In [2]:
def load_Rust_data():
    
    thepath = 'https://raw.githubusercontent.com/math-econ-code/mec_datasets/main/dynamicchoice_Rust/datafiles/'
    
    def getcleandata(name,nrow):
        filepath = thepath+name+'.asc'
        thearray = np.genfromtxt(filepath, delimiter=None, dtype=float).reshape((nrow, -1), order='F')
        odometer1 = thearray[5, :]  # mileage at first replacement (0 if no replacement)
        odometer2 = thearray[8, :]  # mileage at second replacement (0 if no replacement)

        thearray = thearray[11:, :]
        
        replaced1 = (thearray >= odometer1) * (odometer1 > 0)  # replaced once
        replaced2 = (thearray >= odometer2) * (odometer2 > 0)  # replaced twice
        
        running_odometer = np.floor((thearray- odometer1 * replaced1 + (odometer1-odometer2)*replaced2 ) / 5000).astype(int)
        T,B = thearray.shape
        replact = np.array([[ (1 if (replaced1[t+1,b] and not replaced1[t,b]) or (replaced2[t+1,b] and not replaced2[t,b])  else 0) for b in range(B)]
                for t in range(T-1)]+
                        [[0 for b in range(B)]])
        increment = np.array([[ 0 for b in range(B)]]+
                            [[ running_odometer[t+1,b] - running_odometer[t,b] * (1-replact[t,b]) for b in range(B)] for t in range(T-1)])

        return np.block([[running_odometer.reshape((-1,1) ),replact.reshape((-1,1) ), increment.reshape((-1,1) )]])
    
    return np.vstack([getcleandata(name,nrow) for (name, nrow) in [ ('g870',36),('rt50',60),('t8h203',81),('a530875',128) ]])

Alternatively, one can get the `load_Rust_data` function from the `mec` package using: 

In [3]:
# !pip install mec
# from mec.data import load_Rust_data

We may now build the model used by Rust in the original 1987 paper. This model states that: 
* the operating cost (minus the payoff if $y=0$) is equal to $ 0.001 \lambda_0 x $, and 
* the repair cost (minus the payoff if $y=1$) is equal to $\lambda_1$.

Hence $K=2$, and $\lambda_0$ is the per period incremental operating cost, while $\lambda_1$ is the repair cost.

Further, if one lets the bus run, that is if $y=0$, there is a probability 
* $\theta_{30}$ of transitioning from state $x$ to state $x$
* $\theta_{31}$ of transitioning from state $x$ to state $x+1$
* $\theta_{32}$ of transitioning from state $x$ to state $x+2$

and if $y=1$, the bus returns to the initial state $x=0$ and then runs, so there is a probability 
* $\theta_{30}$ of transitioning from state $x$ to state $0$
* $\theta_{31}$ of transitioning from state $x$ to state $1$
* $\theta_{32}$ of transitioning from state $x$ to state $2$

The following code estimates $\theta_{30}$, $\theta_{31}$, and $\theta_{32}$, and sets up the payoff function $\phi_{xyk}$.

In [4]:
thedataset = load_Rust_data()

def build_rust_model_primitives(thedata, X=90):
    theta30 = (thedataset[:,2]==0).mean()
    theta31 = (thedataset[:,2]==1).mean()
    theta32 = 1-theta30 - theta31
    P_xp_x = theta30 * np.eye(X) + np.diag([theta31]*(X-1),-1) + np.diag([theta32]*(X-2),-2)
    P_xp_x[X-1,X-2] += theta32
    P_xp_x[X-1,X-1] = 1
    P_xp_x_y = np.zeros((X,X,2))
    P_xp_x_y[:,:,0] = P_xp_x
    P_xp_x_y[:,:,1] = P_xp_x[:,0][:,None] 
    #
    phi_x_y_k = np.zeros((X,2,2))
    phi_x_y_k[:,1,0] = -1.0 # if repair, replacement cost is lambda_0
    phi_x_y_k[:,1,1] = 0.0 # if repair, replacement cost does not depend on mileage 
    phi_x_y_k[:,0,0] = 0.0 # if no repair, operating cost independent of the repair cost
    phi_x_y_k[:,0,1] = -0.001 * np.arange(X,dtype = np.float64) # if no repair, operating cost proportional to mileage   
    muhat_x_y = np.zeros((X,2))
    for x in range(X):
        muhat_x_y[x,0] = ((thedata[:,0] ==x) & (thedata[:,1]==0)).sum()
        muhat_x_y[x,1] = ((thedata[:,0] ==x) & (thedata[:,1]==1)).sum()
    return sp.csr_matrix(P_xp_x_y.reshape((X,-1))), phi_x_y_k, muhat_x_y.flatten()


We define the important elements of the model, which encode the log-likelihood:

In [5]:
X,Y,K = 90,2,2
P_xprime_xy,phi_x_y_k,muhat_xy = build_rust_model_primitives(thedataset,X)
beta = 0.9999
rel_tol = 1e-5

# Computing:
phi_xy_k = phi_x_y_k.reshape((-1,K))
SigmaY = sp.kron( sp.eye(X),np.ones((1,Y))).tocsr()
P_xy_xprime = P_xprime_xy.T.tocsr()
Psi = (beta* P_xy_xprime - SigmaY.T.tocsr())
phi_x_y_k = phi_xy_k.reshape((X,Y,K))
#tol_lambda = rel_tol* np.abs(phi_x_y_k).max()*muhat_xy.sum()

def fu_x(lambda_k, tol_u=1e-6,max_it = 1000000):
    u_x = np.zeros(X)
    phi_x_y = (phi_xy_k @ lambda_k).reshape((X,Y))
    for s in range(max_it):
        m_x = (phi_x_y + beta * (P_xy_xprime @ u_x).reshape((X,Y))).max(axis = 1)
        unew_x = m_x + np.log( np.exp(phi_x_y + beta * (P_xy_xprime @ u_x).reshape((X,Y)) -m_x[:,None] ).sum(axis = 1) )
        if np.abs(u_x - unew_x).max()<tol_u:
            break
        u_x = unew_x
    if s==max_it-1:
        print(s)
    return(u_x)

def fdu_x(lambda_k, u_x = None):
    if u_x is None:
        u_x = fu_x(lambda_k).flatten()
    pi_xy = np.exp( phi_xy_k @ lambda_k + Psi @ u_x)
    return - sp.linalg.spsolve(SigmaY @ sp.diags(pi_xy) @ Psi , SigmaY @ sp.diags(pi_xy) @ phi_xy_k)

def hessF(lambda_k, u_x = None):
    if u_x is None:
        u_x = fu_x(lambda_k).flatten()
    pi_xy = np.exp( phi_xy_k @ lambda_k + Psi @ u_x)
    dlogpi = phi_xy_k + Psi @ fdu_x(lambda_k,u_x)
    XY = X*Y
    TTT = sp.csr_matrix( ([1]*XY,(list(range(XY)), [a*(XY+1) for a in range(XY)])),shape = (XY,XY*XY) )
    Tdlogpisquare =  TTT @ sp.kron(dlogpi,dlogpi )
    d2u = - sp.linalg.spsolve( SigmaY @ sp.diags(pi_xy) @ Psi , (SigmaY @ sp.diags(pi_xy) @  Tdlogpisquare).todense() )
    return ( muhat_xy.T @ Psi @ d2u).reshape((K,K))

def F(lambda_k, grad): # sample log-likelihood
    u_x = fu_x(lambda_k)
    if grad.size > 0:
        grad[:] = muhat_xy @ ( Psi @ fdu_x(lambda_k,u_x) + phi_xy_k)
    return  muhat_xy @ ( phi_xy_k @ lambda_k + Psi @ u_x)



We run the maximum likelihood estimation program using:

In [6]:
opt = nlopt.opt(nlopt.LD_SLSQP, K) 
opt.set_max_objective(F)
opt.set_xtol_rel(1e-4)
x0 = np.zeros(K)
lambdahat_k = opt.optimize(x0)
max_F = opt.last_optimum_value()
print(f"lambdahat: {lambdahat_k}")
print(f"Log-likelihood: {max_F}")
I = muhat_xy.sum()
V = np.diag(muhat_xy) / I - muhat_xy[:,None] @ muhat_xy[None,:] / (I*I)
u_x = fu_x(lambdahat_k)
HinvB = - np.linalg.solve(hessF(lambdahat_k,u_x), (phi_xy_k + Psi @ fdu_x(lambdahat_k,u_x)).T)
print('Nb observations=')
print(I)
print('Cov(lambdahat)=')
print(HinvB @ V @ HinvB.T)


lambdahat: [9.78513363 2.60375824]
Log-likelihood: -300.23752495157146
Nb observations=
8260.0
Cov(lambdahat)=
[[5.35249297e-05 2.49830904e-05]
 [2.49830904e-05 1.62135559e-05]]
